In [ ]:
!gdown 1YAQHOBSmpDfwYeFq1dMQv94DyYPkp7Xc

Downloading...
From: https://drive.google.com/uc?id=1YAQHOBSmpDfwYeFq1dMQv94DyYPkp7Xc
To: /content/alzheimer.csv
100% 2.38M/2.38M [00:00<00:00, 91.9MB/s]


In [ ]:
import pandas as pd
import numpy as np
from sklearn import metrics
from xgboost import XGBClassifier
from sklearn.impute import SimpleImputer
from sklearn.impute import KNNImputer
from sklearn.preprocessing import LabelEncoder
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix

In [ ]:
data=pd.read_csv('/content/alzheimer.csv')
data

,PatientID,Domain,Age,BMI,HeartRate,BloodPressure,Cholesterol,CognitiveTestScore,StressMobilityTestScore,RiskLevel,FamilyHistoryDementia,TreatmentResponse
0,P000001,Orthopedics,62.0,28.09,69.0,115.0,206.0,82.78,59.68,High Risk,No,Stable
1,P000002,Orthopedics,19.0,22.83,78.0,116.0,196.0,76.00,60.13,Low Risk,No,Improved
2,P000003,Pediatrics,18.0,27.40,71.0,116.0,200.0,58.52,68.76,Low Risk,Yes,Deteriorated
3,P000004,Pediatrics,47.0,27.81,78.0,105.0,176.0,83.56,70.71,Medium Risk,Yes,Improved
4,P000005,Neurology,38.0,17.92,69.0,113.0,208.0,74.45,76.36,High Risk,No,Stable
...,...,...,...,...,...,...,...,...,...,...,...,...
29995,P029996,Cardiology,30.0,29.28,65.0,138.0,286.0,80.59,81.70,Medium Risk,No,Improved
29996,P029997,Orthopedics,42.0,34.00,68.0,117.0,190.0,62.90,69.58,Low Risk,No,Improved
29997,P029998,Pediatrics,69.0,22.87,59.0,107.0,212.0,82.72,50.81,Low Risk,No,Stable
29998,P029999,Pediatrics,42.0,29.74,74.0,115.0,191.0,74.63,63.08,High Risk,No,Improved


In [ ]:
data.isnull().sum()

,0
PatientID,0
Domain,200
Age,200
BMI,200
HeartRate,50
BloodPressure,200
Cholesterol,200
CognitiveTestScore,50
StressMobilityTestScore,200
RiskLevel,0


In [ ]:
data=data.drop(columns=['PatientID','Domain'])

In [ ]:
data.isnull().sum()

,0
Age,200
BMI,200
HeartRate,50
BloodPressure,200
Cholesterol,200
CognitiveTestScore,50
StressMobilityTestScore,200
RiskLevel,0
FamilyHistoryDementia,0
TreatmentResponse,0


In [ ]:
def simple_impute_data(data:pd.DataFrame, columns:list, strategy:str):
  imputer = SimpleImputer(strategy=strategy)
  data_copy = data.copy()

  for column in columns:
    data_copy[column] = imputer.fit_transform(data_copy[[column]])
  return data_copy

In [ ]:
data = simple_impute_data(data=data, columns=['Age', 'BMI', 'HeartRate', 'BloodPressure', 'Cholesterol', 'CognitiveTestScore', 'StressMobilityTestScore'], strategy='mean')

In [ ]:
data.isnull().sum()

,0
Age,0
BMI,0
HeartRate,0
BloodPressure,0
Cholesterol,0
CognitiveTestScore,0
StressMobilityTestScore,0
RiskLevel,0
FamilyHistoryDementia,0
TreatmentResponse,0


In [ ]:
data.head(3)

,Age,BMI,HeartRate,BloodPressure,Cholesterol,CognitiveTestScore,StressMobilityTestScore,RiskLevel,FamilyHistoryDementia,TreatmentResponse
0,62.0,28.09,69.0,115.0,206.0,82.78,59.68,High Risk,No,Stable
1,19.0,22.83,78.0,116.0,196.0,76.00,60.13,Low Risk,No,Improved
2,18.0,27.40,71.0,116.0,200.0,58.52,68.76,Low Risk,Yes,Deteriorated


In [ ]:
def label_data(data:pd.DataFrame, columns:list):
  encoder = LabelEncoder()
  data_copy = data.copy()

  for column in columns:
    data_copy[column] = encoder.fit_transform(data_copy[[column]].astype(str).values.ravel())

    # if 'nan' in encoder.classes_:
    #   data_copy.loc[data_copy[column] == -1, column] = np.nan
  return data_copy

In [ ]:
data = label_data(data=data, columns=['RiskLevel', 'FamilyHistoryDementia', 'TreatmentResponse'])

In [ ]:
data.head(3)

,Age,BMI,HeartRate,BloodPressure,Cholesterol,CognitiveTestScore,StressMobilityTestScore,RiskLevel,FamilyHistoryDementia,TreatmentResponse
0,62.0,28.09,69.0,115.0,206.0,82.78,59.68,0,0,2
1,19.0,22.83,78.0,116.0,196.0,76.00,60.13,1,0,1
2,18.0,27.40,71.0,116.0,200.0,58.52,68.76,1,1,0


In [ ]:
x=data.drop(columns='RiskLevel',axis=1)
y=data['RiskLevel']

In [ ]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2)

In [ ]:
model=XGBClassifier(max_depth=10, min_child_weight=1, n_estimators=100, n_jobs=2, learning_rate=0.9)
model.fit(x_train,y_train)
pred=model.predict(x_test)

In [ ]:
print(metrics.accuracy_score(y_test,pred))

0.3413333333333333


In [ ]:
print(confusion_matrix(y_test,pred))

[[657 649 651]
 [673 778 661]
 [603 715 613]]


In [ ]:
print(classification_report(y_test,pred))

              precision    recall  f1-score   support

           0       0.34      0.34      0.34      1957
           1       0.36      0.37      0.37      2112
           2       0.32      0.32      0.32      1931

    accuracy                           0.34      6000
   macro avg       0.34      0.34      0.34      6000
weighted avg       0.34      0.34      0.34      6000



In [ ]:
classifier=DecisionTreeClassifier(max_depth=5)
classifier.fit(x_train,y_train)
p=classifier.predict(x_test)

In [ ]:
print(classification_report(y_test,p))

              precision    recall  f1-score   support

           0       0.33      0.07      0.11      1957
           1       0.37      0.50      0.42      2112
           2       0.32      0.44      0.37      1931

    accuracy                           0.34      6000
   macro avg       0.34      0.34      0.30      6000
weighted avg       0.34      0.34      0.30      6000



In [ ]:
print("Accuracy:", metrics.accuracy_score(y_test, p))
print(confusion_matrix(y_test, p))

Accuracy: 0.3413333333333333
[[ 134  898  925]
 [ 132 1055  925]
 [ 143  929  859]]
